In [1]:
import sys
import torch
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")
from ronek2 import env
from ronek2 import const
from ronek2.roms import CoBRAS
from ronek2 import backend as bkd
from ronek2.systems import ArgonCR
from ronek2.systems.equilibrium import Equilibrium

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/CR_Ar/database"

In [5]:
model = ArgonCR(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  use_coll_int_fit=True,
  isothermal=False
)
model.set_output(max_mom=2, linear=True)

In [6]:
sol_ref = "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/StS/argon/testcase/output_box"

In [ ]:
columns = "t X_em X_Ar X_Arp Th Te rho p n E".split()
box = np.loadtxt(sol_ref + "/box.dat")
box = pd.DataFrame(box, columns=columns)
box

In [8]:
t = box["t"].values
rho, Th0, Te0 = [box[k][0] for k in ("rho", "Th", "Te")]

In [9]:
def read_pop(name):
  ni_gi  = np.loadtxt(sol_ref + f"/pop_{name}.dat", comments="&", skiprows=2)[:,1]
  levels = model.mix.species[name].lev
  nb_pts = len(ni_gi) / len(levels["g"])
  ni_gi  = np.stack(np.split(ni_gi, nb_pts, axis=0), axis=0)
  return ni_gi * levels["g"].numpy()

In [10]:
n = {k: read_pop(k) for k in ("Ar", "Arp")}
n["em"] = (box["X_em"] * box["n"]).values.reshape(-1,1)
n_true = np.hstack([n[k] for k in model.species_order]).T
w_true = model.mix.get_rho(bkd.to_torch(n_true)).numpy() / rho
T_true = box[["Th", "Te"]].values.T

In [ ]:
p = box["p"].values[-1]
T = box["Th"].values[-1]
neq = model.compute_eq_comp(torch.tensor(p), torch.tensor(T)).numpy()
p, T

In [ ]:
y0 = np.zeros(model.nb_eqs)
y0[:model.mix.nb_comp] = n_true[:,0] / const.UNA
y0[-2] = Th0
y0[-1] = Te0 if model.isothermal else model.mix.get_pe(Te=Te0, ne=n_true[-1,0])
y0

In [ ]:
y, runtime = model.solve_fom(t, y0, rho)
runtime

In [14]:
# w_pred = y[:model.mix.nb_comp]
# n_pred = model.mix.get_n(bkd.to_torch(w_pred)).numpy()
n_pred = y[:model.mix.nb_comp] * const.UNA
T_pred = y[model.mix.nb_comp:]
T_pred[-1] = T_pred[-1] if model.isothermal else model.mix.get_Te(pe=T_pred[-1], ne=n_pred[-1])

In [15]:
neq = n_pred[:,-1]
Teq = T_pred[-1,-1]
peq = np.sum(neq) * const.UKB * Teq
neq_pred = model.compute_eq_comp(torch.tensor(peq), torch.tensor(Teq)).numpy()

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_true[i] - T_pred[i])/T_true[i]))
  err.append(ierr)
  plt.loglog(t[1:], T_true[i][1:])
  plt.loglog(t[1:], T_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_true[i] - T_pred[i])/T_true[i]))
  err.append(ierr)
  plt.semilogx(t[1:], T_true[i][1:])
  plt.semilogx(t[1:], T_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_true[i] - n_pred[i])/n_true[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_true[i][1:])
    plt.loglog(t[1:], n_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()

In [19]:
eq = Equilibrium(mixture=model.mix, clipping=True)

In [20]:
neq = n_pred[:,-1]
Teq = T_pred[-1,-1]
peq = np.sum(neq) * const.UKB * Teq

In [ ]:
rhoeq = model.mix.m @ neq
rhoeq.numpy(), rho

In [ ]:
rhoeq.numpy(), Teq, Teq

In [ ]:
eq.from_prim(rhoeq.numpy(), Teq, Teq)

In [ ]:
(eq.from_prim(rhoeq.numpy(), Teq, Teq) - neq)/neq

In [ ]:
stop

In [21]:
rom = CoBRAS(
  system=model,
  quadrature=None,
  path_to_saving="./",
  saving=True,
  verbose=True
)
rom.build_sol_interp(t, y)

In [ ]:
tm = -np.flip(t[:-600])
tm

In [ ]:
g0 = np.zeros_like(model.C[0])
g0[2] = 1.0
y = sp.integrate.solve_ivp(
  fun=rom.adjoint_fun,
  t_span=[tm[0],tm[-1]],
  y0=g0,
  method="LSODA",
  t_eval=tm,
  first_step=1e-14,
  rtol=1e-6,
  atol=1e-10,
  jac=rom.adjoint_jac
).y

In [ ]:
stop

In [ ]:
rom.adjoint_fun(t[-1], model.C[0])

In [23]:
x = model.jac(t[-1], y[:,-1])
np.savetxt("jac.txt", x)

In [22]:
x = rom.adjoint_jac(t[-1], model.C[0])
np.savetxt("jac.txt", x)

In [ ]:
for i, g0 in enumerate(model.C):
  print(f"Solving {i}")
  g = rom.solve_adjoint(np.flip(t), g0)
  print(g)

In [ ]:
plt.plot()